In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor, VotingRegressor, StackingRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from tensorflow.python.client import device_lib

In [2]:
import pandas as pd
csv1 = "solar_dataset.csv"
df = pd.read_csv(csv1)
df.head(2)

,Timestamp,Battery_Level_Avg,Relative_Humidity_Avg,Rain_Accu_mm,Air_Temperature_Avg,Air_Temperature_Max,Air_Temperature_Min,Wind_Speed_Avg,Wind_Speed_Max,Wind_Speed_Sdev,...,Solar_Azimuth_Avg,Direct_Theoretical_Radiation_Avg,Pyrheliometer_Error_1_Avg,Geotrac3K_Status_Logic_OR_Status,Access_Control_Logic_OR_Status,Global_Energy_1 _Avg,Diffuse_Energy_1_Avg,Direct_Energy_1_Avg,Day,Power
0,2/1/2018 0:00,13.43693,65.072182,0.0,18.253019,18.313231,18.213110,1.2299,1.960,0.477567,...,254.903107,0.0,0.0,0,0,0.000003,0.000002,0.000031,32,20.599508
1,2/1/2018 0:01,13.43391,65.232132,0.0,18.249380,18.285450,18.216471,1.5043,2.058,0.267083,...,254.903107,0.0,0.0,0,0,0.000002,0.000002,0.000031,32,20.599615


In [3]:
df=df.drop(['Timestamp','Solar_Elevation_Avg','Day', 'Direct_Radiation_1_Integr', 'Direct_Radiation_1_Ins','Solar_Azimuth_Ins','Solar_Elevation_Ins',
            'Inclined_Solar_Radiation_Integr','Inclined_Solar_Radiation_Ins', 'Horizontal_Solar_Radiation_Integr','Horizontal_Solar_Radiation_Ins',
            'Diffuse_Radiation_Integr','Diffuse_Radiation_Ins','Global_Radiation_1_Integr',  'Global_Radiation_1_Ins',  'Wind_Direction_Max', 'Wind_Direction_Sdev', 'Wind_Speed_Max', 'Wind_Speed_Sdev','Air_Temperature_Max', 'Air_Temperature_Min',
            'Battery_Level_Avg','Pyrheliometer_Error_1_Avg','Geotrac3K_Status_Logic_OR_Status',  'Access_Control_Logic_OR_Status'],axis=1)
df.rename(columns = {' Horizontal_Solar_Radiation_Avg':'Horizontal_Solar_Radiation_Avg'}, inplace = True)
df.rename(columns = {'Direct_Radiation_1_Avg':'Direct_Radiation_Avg','Global_Energy_1 _Avg':'Global_Energy_Avg','Direct_Energy_1_Avg':'Direct_Energy_Avg','Global_Radiation_1_Avg':'Global_Radiation_Avg','Diffuse_Energy_1_Avg':'Diffuse_Energy_Avg'}, inplace = True)

In [4]:
column_to_drop=['Dew_Point_Avg',
 'Rain_Accu_mm',
 'Atmospheric_Pressure_QNH_Avg',
 'Atmospheric_Pressure_QFE_Avg',
 'Direct_Theoretical_Radiation_Avg',
 'Horizontal_Solar_Radiation_Avg',
 'Global_Energy_Avg',
 'Diffuse_Energy_Avg',
 'Direct_Energy_Avg',
 'Wind_Direction_Avg']
df=df.drop(column_to_drop,axis=1)

In [5]:
df.columns

Index(['Relative_Humidity_Avg', 'Air_Temperature_Avg', 'Wind_Speed_Avg',
       'Global_Radiation_Avg', 'Diffuse_Radiation_Avg', 'Direct_Radiation_Avg',
       'Inclined_Solar_Radiation_Avg', 'Solar_Azimuth_Avg', 'Power'],
      dtype='object')

In [6]:
df = df[df['Power'] >= 500] 

In [7]:
df.shape

(92684, 9)

In [8]:
def split_data(df):
    X = df.drop(columns=['Power'])
    pac = df['Power'].values
    train_size = int(0.8*len(pac)) ##80% train 
    X_train ,  X_test  =   X.iloc[:train_size,:],   X.iloc[train_size:,:];
    pac_train, pac_test= pac[:train_size],   pac[train_size:];
    #filter out pac=0
    ix = pac_test > 0
    pac_test = pac_test[ix]
    X_test = X_test[ix]
    return X_train, X_test, pac_train, pac_test;

X_train, X_test, y_train, y_test = split_data(df)

In [9]:
RF_f=RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt', min_samples_split=4, n_estimators=1700)
RF_f.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt',
                      min_samples_split=4, n_estimators=1700)

In [10]:
XT=ExtraTreesRegressor(n_estimators=1000,random_state=73)
XT.fit(X_train,y_train)

ExtraTreesRegressor(n_estimators=1000, random_state=73)

In [11]:
Ada_f=AdaBoostRegressor(learning_rate=0.03, loss='exponential', n_estimators=2300, random_state=73)
Ada_f.fit(X_train,y_train)


AdaBoostRegressor(learning_rate=0.03, loss='exponential', n_estimators=2300,
                  random_state=73)

In [12]:
import pickle
path = "reduced_data_models/"

models={"Ada_f":Ada_f,"XT":XT,"RF_f":RF_f}

for key,model in models.items():
 
  filename = path + str(key) + ".sav"
  pickle.dump(model, open(filename, 'wb'))
  print(key)

Ada_f
XT
RF_f


In [10]:
GB_f=GradientBoostingRegressor(max_depth=10, max_features='sqrt',
                          min_samples_split=12, n_estimators=1500)
GB_f.fit(X_train,y_train)


GradientBoostingRegressor(max_depth=10, max_features='sqrt',
                          min_samples_split=12, n_estimators=1500)

In [11]:
HGB_f=HistGradientBoostingRegressor(learning_rate=0.02, loss='least_absolute_deviation', max_depth=40, max_iter=750, min_samples_leaf=2, random_state=73)
HGB_f.fit(X_train,y_train)

HistGradientBoostingRegressor(learning_rate=0.02,
                              loss='least_absolute_deviation', max_depth=40,
                              max_iter=750, min_samples_leaf=2,
                              random_state=73)

In [12]:
XGB=XGBRegressor(n_estimators=1200,learning_rate=0.05,random_state=73)
XGB.fit(X_train,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1200, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=73,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
LGBg_f=LGBMRegressor(objective='regression',num_leaves=5,  learning_rate=0.05, n_estimators=720, max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
LGBg_f.fit(X_train,y_train)

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
              feature_fraction=0.2319, feature_fraction_seed=9,
              learning_rate=0.05, max_bin=55, min_data_in_leaf=6,
              min_sum_hessian_in_leaf=11, n_estimators=720, num_leaves=5,
              objective='regression')

In [16]:
import pickle
import joblib
path = "reduced_data_models/"
model={}
models={"HGB_f":HGB_f,"XGB":XGB,"GB_f":GB_f,"LGBg_f":LGBg_f}

for key,model in models.items():
  filename = path + str(key) + ".sav" 
  joblib.dump(model, filename)  
  print(key)

HGB_f
XGB
GB_f
LGBg_f


In [10]:
import pickle
import joblib
from sklearn.metrics import mean_squared_error
from math import sqrt



path = "reduced_data_models/"

models=["HGB_f","XGB","GB_f","LGBg_f","XT","RF_f","Ada_f"]
#model_names=["Random Forest Regressor"]

rmse_dict={}

for model in models:
 
  filename = path + str(model) + ".sav" 
  m=joblib.load(filename, mmap_mode=None)  
  
  y_predicted=m.predict(X_test)


  rmse = sqrt(mean_squared_error(y_test, y_predicted))
  print(rmse)
  rmse_dict[model]=rmse
  print(model)
  m=0



344.99344072381683
HGB_f
352.66521345638074
XGB
410.3264568288198
GB_f
351.3868008347868
LGBg_f
358.79608343077206
XT
367.44581701268083
RF_f
1028.9803241826203
Ada_f


In [11]:
model_name_list=list(rmse_dict.keys())#["Stacking Regressor","Voting Regressor","nb","HGB_f","XGB","GB_f","LGBg_f","XT","RF_f","Ada_f"]
results = pd.DataFrame(columns=['RMSE'], index = model_name_list)
for i in range(0, len(model_name_list)):
  results.iloc[i, :] = [rmse_dict[model_name_list[i]]]
results
results.sort_values("RMSE")

,RMSE
HGB_f,344.993441
LGBg_f,351.386801
XGB,352.665213
XT,358.796083
RF_f,367.445817
GB_f,410.326457
Ada_f,1028.980324


In [12]:
XT=ExtraTreesRegressor(n_estimators=1000,random_state=73)


HGB_f=HistGradientBoostingRegressor(learning_rate=0.02,
                              loss='least_absolute_deviation', max_depth=40,
                              max_iter=750, min_samples_leaf=2,
                              random_state=73)
XGB=XGBRegressor(n_estimators=1200,learning_rate=0.05,random_state=73)

LGBg_f=LGBMRegressor(objective='regression',num_leaves=5,  learning_rate=0.05, n_estimators=720, max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [13]:
estimators=[('lgbgf',LGBg_f),('xt',XT),('xgb',XGB),('hgb',HGB_f)]
Stack=StackingRegressor(estimators=estimators,final_estimator=HGB_f)
Stack.fit(X_train,y_train)


[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11

StackingRegressor(estimators=[('lgbgf',
                               LGBMRegressor(bagging_fraction=0.8,
                                             bagging_freq=5, bagging_seed=9,
                                             feature_fraction=0.2319,
                                             feature_fraction_seed=9,
                                             learning_rate=0.05, max_bin=55,
                                             min_data_in_leaf=6,
                                             min_sum_hessian_in_leaf=11,
                                             n_estimators=720, num_leaves=5,
                                             objective='regression')),
                              ('xt',
                               ExtraTreesRegressor(n_estimators=1000,
                                                   random_state=73)),
                              ('xgb...
                                            validate_parameters=None,
                           

In [14]:
import pickle
import joblib
path = "reduced_data_models/"

models={"Stack_2":Stack}

for key,model in models.items():
 
  filename = path + str(key) + ".sav"
  #pickle.dump(model, open(filename, 'wb'))
  #filename = 'finalized_model.sav'
  joblib.dump(model, filename)  
  print(key)


Stack_2


In [15]:
Vote=VotingRegressor([('lgbgf',LGBg_f),('xt',XT),('xgb',XGB),('hgb',HGB_f)])
Vote.fit(X_train,y_train)

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


VotingRegressor(estimators=[('lgbgf',
                             LGBMRegressor(bagging_fraction=0.8, bagging_freq=5,
                                           bagging_seed=9,
                                           feature_fraction=0.2319,
                                           feature_fraction_seed=9,
                                           learning_rate=0.05, max_bin=55,
                                           min_data_in_leaf=6,
                                           min_sum_hessian_in_leaf=11,
                                           n_estimators=720, num_leaves=5,
                                           objective='regression')),
                            ('xt',
                             ExtraTreesRegressor(n_estimators=1000,
                                                 random_state=73)),
                            ('xgb',...
                                          n_estimators=1200, n_jobs=None,
                                          num_par

In [16]:
import pickle
path = "reduced_data_models/"

models={"Vote_2":Vote}

for key,model in models.items():
 
  filename = path + str(key) + ".sav" 
  joblib.dump(model, filename)  
  print(key)

Vote_2


In [1]:
import joblib
path = "reduced_data_models/"


models=["Vote_2"]
rmse_dict={}

for model in models:
 
  filename = path + str(model) + ".sav" 
  m=joblib.load(filename, mmap_mode=None)  
  print(model)
    

Vote_2


In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt
power_df = pd.DataFrame(columns=['Power_Actual','Power_Predicted'])

y_predicted=m.predict(X_test)
power_df = power_df.append({'Timestamp':copy_df['Tim,'Power_Actual':copy_df["Power"].iloc[i], 'Power_Calculated':p}, ignore_index=True)


rmse = sqrt(mean_squared_error(y_test, y_predicted))
print(rmse)
rmse_dict[model]=rmse
print(model)
m=0

IndexError: index 20000 is out of bounds for axis 0 with size 18537

340.62451559534594
Vote_2